In [1]:
import requests
import json
import pandas as pd
import numpy as np
import ift6758
from pathlib import Path

In [2]:
import os
from pathlib import Path
import logging
from flask import Flask, jsonify, request, abort, g
import xgboost
import requests
import pandas as pd
from comet_ml import API
import joblib

import sklearn
import ift6758

In [3]:
try:
    hey = requests.get("http://0.0.0.0:5000/hello")
except Exception as e:
    print(e)

In [4]:
hey.text

'Hello!'

In [5]:
logs = requests.get("http://0.0.0.0:5000/logs")
logs.json()[-5:]

['2022-12-19 16:37:50,413;DEBUG;Input DataFrame shape:(50, 27)',
 '2022-12-19 16:37:50,425;DEBUG;Prediction DataFrame shape:(50,)',
 '2022-12-19 16:37:50,425;INFO;Prediction end',
 '2022-12-19 16:38:14,446;INFO;Accessed page /hello - Hello!',
 '2022-12-19 16:38:14,711;INFO;Accessed page /logs']

In [12]:
default_model = {
    'workspace': 'ift-6758-projet-quipe-13',
    'model': 'xgboost-base-all-features',
    'version': '1.0.0'
}

In [13]:
model = requests.post("http://0.0.0.0:5000/download_registry_model", json=default_model)

In [14]:
model.text

'"Updated from CometML api."\n'

In [15]:
model

<Response [200]>

In [16]:
df = pd.DataFrame(np.random.randn(5266, 29))

In [17]:
X_dict = {}
X_values = df.values.tolist()
X_dict['values'] = X_values

In [18]:
list(X_dict.keys())[0]

'values'

In [19]:
pred = requests.post("http://0.0.0.0:5000/predict", json=X_dict)

In [20]:
pred.json()[:10]

[0.5804065465927124,
 0.0640469342470169,
 0.4480399489402771,
 0.37254077196121216,
 0.7314621806144714,
 0.445902556180954,
 0.4519246816635132,
 0.44373998045921326,
 0.36011210083961487,
 0.660983145236969]

In [21]:
df = pd.DataFrame(np.random.randn(5266, 27))

In [22]:
X_dict = {}
X_values = df.values.tolist()
X_dict['values'] = X_values

In [23]:
pred = requests.post("http://0.0.0.0:5000/predict", json=X_dict)

In [24]:
pred.json()[:10]

JSONDecodeError: Expecting value: line 1 column 1 (char 0)

In [ ]:
df

In [27]:
a = np.array([np.nan])

In [28]:
xd = pd.DataFrame(a)

In [29]:
xd

,0
0,NaN


In [38]:
if xd.values is None:
    print('hello')

In [41]:
xd.values == npnp.nan

array([[False]])

In [22]:
Y = None

In [24]:
if Y is None:
    print('hi')

hi


In [26]:
features = [
                'periodTimeSec', 'period', 'coordinate_x', 'coordinate_y',
                'distanceFromGoal', 'shotAngle', 'lastEventCoord_x', 'lastEventCoord_y',
                'timeDifference', 'distanceDifference', 'rebound',
                'shotAngleDifference', 'speed', 'Backhand', 'Deflected', 'Slap Shot',
                'Snap Shot', 'Tip-In', 'Wrap-around', 'Wrist Shot', 'BLOCKED_SHOT',
                'FACEOFF', 'GIVEAWAY', 'GOAL', 'HIT', 'MISSED_SHOT', 'SHOT',
                'TAKEAWAY', 'PENALTY'
            ]

In [27]:
xxdd = pd.DataFrame(np.nan, index=[0], columns=features)

In [28]:
xxdd

,periodTimeSec,period,coordinate_x,coordinate_y,distanceFromGoal,shotAngle,lastEventCoord_x,lastEventCoord_y,timeDifference,distanceDifference,...,Wrist Shot,BLOCKED_SHOT,FACEOFF,GIVEAWAY,GOAL,HIT,MISSED_SHOT,SHOT,TAKEAWAY,PENALTY
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
